In [1]:
# Solve a multi - commodity flow problem . Two products (’ Pencils ’ and ’Pens ’)
# are produced in 2 cities (’ Detroit ’ and ’Denver ’) and must be sent to
# warehouses in 3 cities (’ Boston ’, ’New York ’, and ’Seattle ’) to
# satisfy demand (’ inflow [h,i] ’).
#
# Flows on the transportation network must respect arc capacity constraints
# (’ capacity [i,j] ’). The objective is to minimize the sum of the arc
# transportation costs (’ cost [i,j] ’).

In [2]:
import gurobipy as gp
from gurobipy import GRB

In [3]:
# Base data
commodities = ['Pencils', 'Pens']
nodes = ['Detroit', 'Denver', 'Boston', 'NewYork', 'Seattle']

arcs, capacity = gp.multidict({
    ('Detroit', 'Boston'): 100,
    ('Detroit', 'NewYork'): 80,
    ('Detroit', 'Seattle'): 120,
    ('Denver', 'Boston'): 120,
    ('Denver', 'NewYork'): 120,
    ('Denver', 'Seattle'): 120 
})

# Cost for triplets commodity-source-destination
cost = {
    ('Pencils', 'Detroit', 'Boston'): 10,
    ('Pencils', 'Detroit', 'NewYork'): 20,
    ('Pencils', 'Detroit', 'Seattle'): 60,
    ('Pencils', 'Denver', 'Boston'): 40,
    ('Pencils', 'Denver', 'NewYork'): 40,
    ('Pencils', 'Denver', 'Seattle'): 30,
    ('Pens', 'Detroit', 'Boston'): 20,
    ('Pens', 'Detroit', 'NewYork'): 20,
    ('Pens', 'Detroit', 'Seattle'): 80,
    ('Pens', 'Denver', 'Boston'): 60,
    ('Pens', 'Denver', 'NewYork'): 70,
    ('Pens', 'Denver', 'Seattle'): 30,
}

# Demand for pairs of commodity-city
inflow = {
    ('Pencils', 'Detroit'): 50,
    ('Pencils', 'Denver'): 60,
    ('Pencils', 'Boston'): -50,
    ('Pencils', 'NewYork'): -50,
    ('Pencils', 'Seattle'): -10,
    ('Pens', 'Detroit'): 60,
    ('Pens', 'Denver'): 40,
    ('Pens', 'Boston'): -40,
    ('Pens', 'NewYork'): -30,
    ('Pens', 'Seattle'): -30,
}

In [5]:
# Create optimization model
m = gp.Model("netflow")

**问题**：`addVars`中的`obj`参数是什么含义？此外，这个模型好像没有设置目标函数。  

**答**：
文档中对此参数的说明如下:
> obj (optional): Objective coefficient for new variable.

设置目标函数有二种方法：
- 一种是用`setObjective()`函数，这是最直观的。
- 第二种方法，直接在 `addvar` 或者 `addvars` 产生变量时，就通过 `obj =`  来指定在目标中的该变量的系数。二个方法用哪一个都可以。如果你不习惯在 `addvars` 里设置，那就忽略 `obj =` 的系数，然后在  `setObjective()` 函数中设置。

In [12]:
# Create variables
flow = m.addVars(commodities, arcs, obj=cost, name="flow")
flow

{('Pencils', 'Detroit', 'Boston'): <gurobi.Var *Awaiting Model Update*>,
 ('Pencils', 'Detroit', 'NewYork'): <gurobi.Var *Awaiting Model Update*>,
 ('Pencils', 'Detroit', 'Seattle'): <gurobi.Var *Awaiting Model Update*>,
 ('Pencils', 'Denver', 'Boston'): <gurobi.Var *Awaiting Model Update*>,
 ('Pencils', 'Denver', 'NewYork'): <gurobi.Var *Awaiting Model Update*>,
 ('Pencils', 'Denver', 'Seattle'): <gurobi.Var *Awaiting Model Update*>,
 ('Pens', 'Detroit', 'Boston'): <gurobi.Var *Awaiting Model Update*>,
 ('Pens', 'Detroit', 'NewYork'): <gurobi.Var *Awaiting Model Update*>,
 ('Pens', 'Detroit', 'Seattle'): <gurobi.Var *Awaiting Model Update*>,
 ('Pens', 'Denver', 'Boston'): <gurobi.Var *Awaiting Model Update*>,
 ('Pens', 'Denver', 'NewYork'): <gurobi.Var *Awaiting Model Update*>,
 ('Pens', 'Denver', 'Seattle'): <gurobi.Var *Awaiting Model Update*>}

In [9]:
# Arc-capacity constraints
flow = m.addConstrs(
    (flow.sum('*', i, j) <= capacity[i, j] for i, j in arcs), "cap"
)

In [13]:
# Equivalent version using Python looping
for i, j in arcs:
    m.addConstr(sum(flow[h, i, j] for h in commodities) <= capacity[i, j],
                name="cap[%s, %s]" % (i, j) 
               )

注意由于`=`有特殊含义，即：被`lhs`、`sense`等关键词征用，因此若要用等式，这里必须用双等号`==`才不会报错

In [14]:
# Flow conservation constraints
m.addConstrs(
    (flow.sum(h, '*', j) + inflow[h, j] == flow.sum(h, j, '*')
    for h in commodities for j in nodes), "node"
)

{('Pencils', 'Detroit'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'Denver'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'NewYork'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Detroit'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Denver'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'NewYork'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>}

In [15]:
# Alternative version:
m.addConstrs(
    (gp.quicksum(flow[h, i, j] for i, j in arcs.select('*', j)) + 
     inflow[h, j] == 
     gp.quicksum(flow[h, j, k] for j, k in arcs.select(j, '*'))
     for h in commodities for j in nodes), "node"
)

{('Pencils', 'Detroit'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'Denver'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'NewYork'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pencils', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Detroit'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Denver'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Boston'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'NewYork'): <gurobi.Constr *Awaiting Model Update*>,
 ('Pens', 'Seattle'): <gurobi.Constr *Awaiting Model Update*>}

In [16]:
m.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 32 rows, 24 columns and 72 nonzeros
Model fingerprint: 0x830ca4fe
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve removed 32 rows and 24 columns
Presolve time: 0.08s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.5000000e+03   0.000000e+00   2.000000e+01      0s
Extra one simplex iteration after uncrush
       1    5.5000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.10 seconds
Optimal objective  5.500000000e+03


In [17]:
if m.status == GRB.OPTIMAL:
    solution = m.getAttr('x', flow)
    for h in commodities:
        print('\nOptimal flows for %s' % h)
        for i, j in arcs:
            if solution[h, i, j] > 0:
                print('%s -> %s: %g' % (i, j, solution[h, i,j ]))


Optimal flows for Pencils
Detroit -> Boston: 50
Denver -> NewYork: 50
Denver -> Seattle: 10

Optimal flows for Pens
Detroit -> Boston: 30
Detroit -> NewYork: 30
Denver -> Boston: 10
Denver -> Seattle: 30
